In [9]:
import pandas as pd
import difflib 
import numpy as np
from fuzzywuzzy import fuzz
# read xlsx file
provider = pd.read_excel('New account list.xlsx', dtype = {"DUNS" : str,"Domain":str,"Name":str})

data_lake = pd.read_excel('Data Lake.xlsx', dtype = {"Master DUNS" : str,"Parent DUNS" : str})
data_lake["Master Account Name"] = data_lake["Master Account Name"].str.lower()
data_lake["Sales Parent Name"] = data_lake["Sales Parent Name"].str.lower()

##Prep provider list:
#Prep the name to be lowercase in order to use difflib library:
#provider["Name"] = provider["Name"].str.lower()
Desired_columns=['Name','Domain','DUNS']
Provided_columns=provider.columns

#To create the empty columns for either Name,Domain,DUNS if the provided list is lack of it:
for i in Desired_columns:
    if i in Provided_columns:
        pass
    elif i not in Provided_columns:
        provider[i]= pd.Series()

provider['Name'] = provider['Name'].astype("string")
provider['Domain'] = provider['Domain'].astype("string")
provider['DUNS'] = provider['DUNS'].astype("string")
        
provider['Name'] = provider['Name'].str.lower()        
####STANDARDIZE THE LIST MATCH PROCESS FIRST-THE ORDER OF MATCH: DUNS NUMBER> DOMAIN> NAME
####MATCH WITH DUNS NUMBER FIRST
#Seperate the provided list to 2 seperate databases: data with DUNS number and data where there is no NULL number
provider_w_duns = provider[~provider['DUNS'].isnull()]
provider_without_duns = provider[provider['DUNS'].isnull()]

#Prep the database with duns number
provider_w_duns['DUNS'] = provider_w_duns['DUNS'].apply(lambda i: '{0:0>9}'.format(i))

# left join with provider database with database
DUNS_process = provider_w_duns.merge(data_lake, left_on = ['DUNS','DUNS'], right_on = ['Master DUNS','Parent DUNS'], how='left')
DUNS_MATCH = DUNS_process[~DUNS_process["SFDC Master Id"].isnull()]
DUNS_NOT_MATCH = DUNS_process[DUNS_process["SFDC Master Id"].isnull()]

if len(DUNS_MATCH.count(axis=1))>0 and len(DUNS_MATCH[(DUNS_MATCH['Name'].notnull())].count(axis=1))>0:
    DUNS_MATCH_wthname = DUNS_MATCH[~DUNS_MATCH["Name"].isnull()]
    DUNS_MATCH_withoutname = DUNS_MATCH[DUNS_MATCH["Name"].isnull()]
    DUNS_MATCH_wthname['Ratio'] = DUNS_MATCH_wthname.apply(lambda x: fuzz.ratio(x['Name'], x['Master Account Name']), axis=1)
    idx = DUNS_MATCH_wthname.groupby(['Name'])['Ratio'].transform(max) == DUNS_MATCH_wthname['Ratio']
    DUNS_MATCH_wthname=DUNS_MATCH_wthname[idx]
    DUNS_MATCH_wthname=DUNS_MATCH_wthname.drop_duplicates(subset=['Name','Ratio'], keep='first')
    DUNS_MATCH_wthname.drop('Ratio', axis=1, inplace = True)
    DUNS_MATCH = pd.concat([DUNS_MATCH_wthname, DUNS_MATCH_withoutname], ignore_index = True)

    
if len(DUNS_MATCH.count(axis=1))>0 and len(DUNS_MATCH[(DUNS_MATCH['Name'].notnull())].count(axis=1))==0:
    DUNS_MATCH=DUNS_MATCH.drop_duplicates(subset=['DUNS'], keep='first')



#Sort out the DUNS MATCH, first find the correlation between the provided name and master account name:
#####AFTER MATCH WITH DUNS NUMBER, THE REMAINING WILL BE MATCHED BY Domain category
not_finished=pd.concat([provider_without_duns, DUNS_NOT_MATCH], ignore_index = True)

#Seperate this remaining_domain list to 2 seperate databases: data wth Domain and data where is no Domain:
remaining_w_domain = not_finished[~not_finished['Domain'].isnull()]
remaining_without_domain = not_finished[not_finished['Domain'].isnull()]

##Before merging drop all the datalake columns in remaining_w_domain
dl_columns=data_lake.columns
remaining_w_domain.drop(dl_columns, axis=1, inplace = True)

#left join remaining_w_domain with the data lake
Domain_process = remaining_w_domain.merge(data_lake, left_on = ['Domain','Domain'], right_on = ['Master Domain','Parent Domain'], how='left')
DOMAIN_MATCH = Domain_process[~Domain_process["SFDC Master Id"].isnull()]
Domain_NOT_MATCH = Domain_process[Domain_process["SFDC Master Id"].isnull()]

#Sort out the DUNS MATCH, first find the correlation between the provided name and master account name:

if len(DOMAIN_MATCH.count(axis=1))>0 and len(DOMAIN_MATCH[(DOMAIN_MATCH['Name'].notnull())].count(axis=1))>0:
    DOMAIN_MATCH_wthname = DOMAIN_MATCH[~DOMAIN_MATCH["Name"].isnull()]
    DOMAIN_MATCH_withoutname = DOMAIN_MATCH[DOMAIN_MATCH["Name"].isnull()]
    DOMAIN_MATCH_wthname['Ratio'] = DOMAIN_MATCH_wthname.apply(lambda x: fuzz.ratio(x['Name'], x['Master Account Name']), axis=1)
    idx = DOMAIN_MATCH_wthname.groupby(['Name'])['Ratio'].transform(max) == DOMAIN_MATCH_wthname['Ratio']
    DOMAIN_MATCH_wthname=DOMAIN_MATCH_wthname[idx]
    DOMAIN_MATCH_wthname=DOMAIN_MATCH_wthname.drop_duplicates(subset=['Name','Ratio'], keep='first')
    DOMAIN_MATCH_wthname.drop('Ratio', axis=1, inplace = True)
    DOMAIN_MATCH = pd.concat([DOMAIN_MATCH_wthname, DOMAIN_MATCH_withoutname], ignore_index = True)
    
if len(DOMAIN_MATCH.count(axis=1))>0 and len(DOMAIN_MATCH[(DOMAIN_MATCH['Name'].notnull())].count(axis=1))==0:
    DOMAIN_MATCH=DOMAIN_MATCH.drop_duplicates(subset=['Domain'], keep='first')
        

C:\Users\ado\AppData\Local\Temp\ipykernel_18380\3547326811.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  provider[i]= pd.Series()
C:\Users\ado\AppData\Local\Temp\ipykernel_18380\3547326811.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  provider[i]= pd.Series()


In [10]:
###AFTER MATCH WITH DOMAIN NUMBER, THE REMAINING WILL BE MATCHED BY NAME
# combining all provider without match in data lake,data do not match with duns and data without duns
not_finished = pd.concat([remaining_without_domain, Domain_NOT_MATCH], ignore_index = True)
error_provider = not_finished[~not_finished['Name'].isnull()]
no_name = not_finished[not_finished['Name'].isnull()]

# fuzzy lookup
#Look up closet value to name in master account name and sales parent name:
if len(error_provider.count(axis=1))>0:
    error_provider['match_w_Master_Account_Name'] = error_provider['Name'].apply(lambda x: difflib.get_close_matches(x, data_lake["Master Account Name"], cutoff = 0.9))
    error_provider = error_provider.explode('match_w_Master_Account_Name')
    error_provider['match_w_Master_Account_Name'] = error_provider['match_w_Master_Account_Name'].astype("str") 
    error_provider['MAN_Ratio'] = error_provider.apply(lambda x: fuzz.ratio(x['Name'], x['match_w_Master_Account_Name']), axis=1)
    idx = error_provider.groupby(['Name'])['MAN_Ratio'].transform(max) == error_provider['MAN_Ratio']
    error_provider=error_provider[idx]
    error_provider=error_provider.drop_duplicates(subset=['Name','MAN_Ratio'], keep='first')
    #error_provider.drop('MAN_Ratio', axis=1, inplace = True)

#Look up closet value to name in master account name and sales parent name:
    error_provider['match_w_Sales Parent Name'] = error_provider['Name'].apply(lambda x: difflib.get_close_matches(x, data_lake["Sales Parent Name"], cutoff = 0.9))
    error_provider = error_provider.explode('match_w_Sales Parent Name')
    error_provider['match_w_Sales Parent Name'] = error_provider['match_w_Sales Parent Name'].astype("str") 
    error_provider['SPN_Ratio'] = error_provider.apply(lambda x: fuzz.ratio(x['Name'], x['match_w_Sales Parent Name']), axis=1)
    idx = error_provider.groupby(['Name'])['SPN_Ratio'].transform(max) == error_provider['SPN_Ratio']
    error_provider=error_provider[idx]
    error_provider=error_provider.drop_duplicates(subset=['Name','SPN_Ratio'], keep='first')
    #error_provider.drop('SPN_Ratio', axis=1, inplace = True)

#Divide database to 4 databases: one name can be looked up and one name cannot be looked up:MAN_SPN_avail, MAN_avail_SPN_non_avail, MAN_nonavail_SPN_avail is available MATCH
    MAN_SPN_avail=error_provider[(error_provider['match_w_Master_Account_Name']!='nan') & (error_provider['match_w_Sales Parent Name']!='nan')]
    MAN_avail_SPN_non_avail=error_provider[(error_provider['match_w_Master_Account_Name']!='nan') & (error_provider['match_w_Sales Parent Name']=='nan')]
    MAN_nonavail_SPN_avail=error_provider[(error_provider['match_w_Master_Account_Name']=='nan') & (error_provider['match_w_Sales Parent Name']!='nan')]
    #MAN_SPN_non_avail
    MAN_SPN_non_avail = error_provider[(error_provider['match_w_Master_Account_Name']=='nan') & (error_provider['match_w_Sales Parent Name']=='nan')]

    #concat databases to match with Master Account Name only:
    to_match_w_MAN=pd.concat([MAN_SPN_avail,MAN_avail_SPN_non_avail], ignore_index = True)
    to_match_w_SPN=MAN_nonavail_SPN_avail

    ##Before merging drop all the datalake columns in error_provider_found
    dl_columns=data_lake.columns
    to_match_w_MAN.drop(dl_columns, axis=1, inplace = True)
    to_match_w_SPN.drop(dl_columns, axis=1, inplace = True)

    #Merge database for database where name can be looked up in the data lake:
    #Match Master Account Name and Sales parent name individually:
    MAN_match = to_match_w_MAN.merge(data_lake, left_on = 'match_w_Master_Account_Name', right_on ='Master Account Name', how='left')
    MAN_match=MAN_match.drop_duplicates(subset=['Name'], keep='first')

    #When you match with Sales Parent name they will you many match therefore you need to filter at this step:

    SPN_match = to_match_w_SPN.merge(data_lake, left_on = 'match_w_Sales Parent Name', right_on ='Sales Parent Name', how='left')
    if len(SPN_match.count(axis=1))>0:
        SPN_match['Ratio'] = SPN_match.apply(lambda x: fuzz.ratio(x['Name'], x['Master Account Name']), axis=1)
        idx = SPN_match.groupby(['Name'])['Ratio'].transform(max) == SPN_match['Ratio']
        SPN_match=SPN_match[idx]
        SPN_match=SPN_match.drop_duplicates(subset=['Name','Ratio'], keep='first')
        SPN_match.drop('Ratio', axis=1, inplace = True)

    NAME_MATCH=pd.concat([MAN_match,SPN_match], ignore_index = True)
    #Drop the ['match_w_Master_Account_Name'] and ['match_w_Sales Parent Name']:
    NAME_MATCH.drop(['match_w_Master_Account_Name','match_w_Sales Parent Name'], axis=1, inplace = True)
    NAME_MATCH['MAN_Ratio'] = NAME_MATCH['MAN_Ratio'].astype("int")

    NAME_MATCH=NAME_MATCH.sort_values('MAN_Ratio', axis = 0, ascending = False)
    MAN_SPN_non_avail.drop(['match_w_Master_Account_Name','match_w_Sales Parent Name'], axis=1, inplace = True)

    finished_process = pd.concat([DUNS_MATCH,DOMAIN_MATCH,NAME_MATCH], ignore_index = True)
    not_finished = pd.concat([MAN_SPN_non_avail,no_name], ignore_index = True)
    final_product = pd.concat([finished_process,not_finished], ignore_index = True)
    final_product=final_product.drop_duplicates()
    final_product=final_product.fillna('-')
    
if len(error_provider.count(axis=1))==0:
    finished_process = pd.concat([DUNS_MATCH,DOMAIN_MATCH], ignore_index = True)
    not_finished = pd.concat([remaining_without_domain, Domain_NOT_MATCH], ignore_index = True)
    final_product = pd.concat([finished_process,not_finished], ignore_index = True)
    #final_product=final_product.drop_duplicates()
    final_product=final_product.fillna('-')
    
import os 
output_directory='C:\\Users\\ado\\OneDrive - Softchoice\\Documents\\Python Scripts\\Output' 
final_product.to_excel(os.path.join(output_directory, str('New account list')+str('_COMPLETED.xlsx')))